<a href="https://colab.research.google.com/github/quantmuse/renewable_choropleth/blob/main/renewable_choropleth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install plotly

In [2]:
import plotly.express as px
import pandas as pd
from pandas_datareader import wb

In [3]:
# get country name and ISO id for mapping on choropleth
countries = wb.get_countries()
countries["capitalCity"].replace({"": None}, inplace=True)
countries.dropna(subset=["capitalCity"], inplace=True)
countries = countries[["name", "iso3c"]]
countries = countries[countries["name"] != "Kosovo"]
countries = countries.rename(columns={"name": "country"})

<ipython-input-3-5ee981bde0f9>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  countries["capitalCity"].replace({"": None}, inplace=True)


In [4]:
# specify the indicators you want, then extract the data from worldbank api
indicators = {"EG.ELC.RNWX.KH": "Electricity production from renewable sources, excluding hydroelectric (kWh)"}
# specify years
years_chosen = [2010, 2015] # between these two years

<ipython-input-4-66a5ca5735e6>:4: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = wb.download(


In [ ]:
df = wb.download(
        indicator=(list(indicators)), country=countries["iso3c"], start=years_chosen[0], end=years_chosen[1]
)
df = df.reset_index()
df.year = df.year.astype(int)

# Add country ISO3 id to main df
df = pd.merge(df, countries, on="country")
df = df.rename(columns=indicators)

In [9]:

dff = df[df.year.between(years_chosen[0], years_chosen[1])]
dff = dff.groupby(["iso3c", "country"])[list(indicators.values())[0]].mean()
dff = dff.reset_index()

# generate choropleth
fig = px.choropleth(
      data_frame=dff,
      locations="iso3c",
      color=list(indicators.values())[0],
      scope="world",
      hover_data={"iso3c": False, "country": True},
      )

fig.update_layout(
      geo={"projection": {"type": "natural earth"}},
      margin=dict(l=50, r=50, t=50, b=50),
)

fig.show()